In [1]:
import feedparser
import csv
import pandas as pd
import re

##### Укажите словарь с RSS-лентой, обратите внимание, что не все RSS-ленты имеют доступный для парсинга формат, проверяйте
##### Укажите два вектора по которым будет производить поиск, символ '|' означает "или", если нужен только один вектор, в конце другого добавьте '|' - это будет означать, что вектор ищет все слова. Векторы не чувствительны к регистрам
##### Укажите два пути для файлов, для всех новостей и для определенных новостей

In [2]:
newsurls = {'Kommersant': 'https://www.kommersant.ru/RSS/news.xml',
'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss'} #пример словаря RSS-лент 
                                           #русскоязычных источников

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'ДолЛАР|РубЛ|ЕвРО' #пример таргетов
vector2 = 'ЦБ|СбЕРбАНК|курс'

In [3]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse( rss_url )  
    
def getHeadlines( rss_url ): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions( rss_url ): #функция для получения описания новости
    descriptions = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks( rss_url ): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates( rss_url ): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [4]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend( getHeadlines( url ) )
    
for key,url in newsurls.items():
    alldescriptions.extend( getDescriptions( url ) )
    
for key,url in newsurls.items():
    alllinks.extend( getLinks( url ) )
    
for key,url in newsurls.items():
    alldates.extend( getDates( url ) )

In [5]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [6]:
write_all_news(f_all_news) #все новости

,Title,Description,Links,Publication Date
0,Reuters: Россия готова с апреля восстановить п...,Россия согласилась увеличить скидку для Белору...,https://www.kommersant.ru/doc/4299824,"Mon, 23 Mar 2020 16:55:00 +0300"
1,В Волгограде мужчина убил полицейского во врем...,В Волгограде региональное управление Следствен...,https://www.kommersant.ru/doc/4299825,"Mon, 23 Mar 2020 16:49:00 +0300"
2,Balenciaga и Yves Saint Laurent будут шить мед...,"Группа Kering, один из лидеров мирового рынка ...",https://www.kommersant.ru/doc/4299808,"Mon, 23 Mar 2020 16:24:00 +0300"
3,Счетная палата не будет начинать новые проверк...,Счетная палата (СП) из-за пандемии коронавирус...,https://www.kommersant.ru/doc/4299805,"Mon, 23 Mar 2020 16:22:00 +0300"
4,Владельцы АЗС предупредили о подорожании бензи...,"Независимый топливный союз (НТС), объединяющий...",https://www.kommersant.ru/doc/4299802,"Mon, 23 Mar 2020 16:17:00 +0300"
...,...,...,...,...
587,Россия протянула Италии руку помощи,Италия благодарит Россию за помощь в борьбе с ...,https://www.vesti.ru/doc.html?id=3250274,"Mon, 23 Mar 2020 06:01:00 +0300"
588,Трамп и Пенс сдали анализы на коронавирус и по...,Президенту США Дональду Трампу и вице-президен...,https://www.vesti.ru/doc.html?id=3250275,"Mon, 23 Mar 2020 06:00:00 +0300"
589,В Китае состоялся панда-марафон,Панда-марафон состоялся накануне в Китае на фо...,https://www.vesti.ru/doc.html?id=3250273,"Mon, 23 Mar 2020 05:43:00 +0300"
590,Уральские производители приборов ИВЛ готовы ув...,На Уральском приборостроительном предприятии г...,https://www.vesti.ru/doc.html?id=3250271,"Mon, 23 Mar 2020 05:42:00 +0300"


In [7]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
6,«Уральские авиалинии» с 21 марта отменили пита...,«Уральские авиалинии» с 21 марта временно отме...,https://www.kommersant.ru/doc/4299783,"Mon, 23 Mar 2020 16:06:00 +0300"
19,Официальный курс доллара превысил 80 рублей,Банк России установил на 24 марта официальный ...,https://www.kommersant.ru/doc/4299701,"Mon, 23 Mar 2020 14:47:00 +0300"
24,Курс евро на Мосбирже опустился ниже 86 рублей,Курс евро в ходе торгов на Московской бирже по...,https://www.kommersant.ru/doc/4299668,"Mon, 23 Mar 2020 14:08:00 +0300"
53,Курсы доллара и евро растут в начале торгов на...,На открытии торгов на Московской бирже курсы д...,https://www.kommersant.ru/doc/4299508,"Mon, 23 Mar 2020 10:19:00 +0300"
162,Минэкономики разработало механизм урегулирован...,Минэкономики подготовило проект постановления ...,https://www.kommersant.ru/doc/4299041,"Sat, 21 Mar 2020 18:21:00 +0300"
205,Миронов предложил провести внеочередную индекс...,Лидер «Справедливой России» Сергей Миронов пре...,https://www.kommersant.ru/doc/4298941,"Sat, 21 Mar 2020 06:41:00 +0300"
221,Участники «Евровидения» не смогут выступить в ...,Участники «Евровидения» должны будут представл...,https://www.kommersant.ru/doc/4298926,"Sat, 21 Mar 2020 00:58:00 +0300"
272,Официальный курс доллара превысил 80 рублей,Центробанк повысил официальный курс доллара на...,https://lenta.ru/news/2020/03/23/official_cb/,"Mon, 23 Mar 2020 14:37:00 +0300"
358,Курс доллара резко вырос,На открытии торгов Московской биржи в понедель...,https://lenta.ru/news/2020/03/23/dollar/,"Mon, 23 Mar 2020 10:19:00 +0300"
436,"В Башкирии менеджер банка получила 3,5 года за...","В Башкирии вынесли приговор экс-менеджеру ""Сбе...",https://www.vesti.ru/doc.html?id=3250467,"Mon, 23 Mar 2020 16:24:00 +0300"
